In [1]:
import pandas as pd
from config import *

In [2]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

In [3]:
engine = create_engine(conn2, echo=False) 

In [4]:
Base = automap_base()
Base.prepare(engine, reflect=True)  

In [5]:
from os import listdir

In [ ]:
# text dataframe
text_df = df[['id_str', 'text']]

In [6]:
filepaths = ['csvs/' + f for f in listdir("csvs")]
df = pd.concat(map(pd.read_csv, filepaths))
df = df.drop_duplicates(subset=['id_str'])
len(df)

232058

In [7]:
df = df.reset_index()
df = df.drop(columns=['index'])
df.tail(1)

,created,id_str,text,user_id,screen_name,location,followers_count,user_favourites_count,time_zone,geo_enabled,verified,status_count,geo,coords,retweet_count,tweet_favourite_count,Subjectivity,Polarity,Sentiment
232057,2021-03-15 00:02:57,1371250530123718660,Found my old WHO vaccine cert booklet from whe...,92157908,Juliagoolia1982,Anywhere but here,761,16382,NaN,True,False,13107,NaN,NaN,0,0,0.2,0.1,positive


In [9]:
# text dataframe
text_df = df[['id_str', 'text','Sentiment']]
text_df

,id_str,text,Sentiment
0,1367263516457979910,Shieldk2 Yes! My wife works for Pfizer and has...,neutral
1,1367263515656867840,"oleary_ray: ""I will not be taking the Covid va...",positive
2,1367263515388567563,"CDC's report on J&amp;J COVID-19 vaccine - ""AC...",positive
3,1367263513257930752,ByYourLogic: the Pfizer vaccine makes your dic...,positive
4,1367263503782928384,RobDownenChron: The Archdiocese says the new J...,positive
...,...,...,...
232053,1371250540449988610,f0lake: No i dont think u understand i need to...,neutral
232054,1371250536633208834,PeterHotez: It’s why we might eventually move ...,positive
232055,1371250532568956935,web_rant: mikeallen axios kadiagoba Gosh Mike ...,neutral
232056,1371250531516223488,Reuters: AstraZeneca finds no evidence of incr...,neutral


In [10]:
from textblob import TextBlob

In [12]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [11]:
def Tokenize(txt):
    # converts str to list 
    return TextBlob(txt).words

In [13]:
def RemoveStopWord(lst):
    stop_words = stopwords.words('english')
    no_stopwords = [word for word in lst if word not in stopwords]
    return no_stopwords

In [14]:
from nltk.stem.porter import PorterStemmer 
porter = PorterStemmer()

def Stem(txt):
    return porter.stem(txt)


In [ ]:
from nltk.stem.wordnet import WordNetLemmatizer
lemmat = WordNetLemmatizer()

def Lemmatize(txt):
    return lemmat.lemmatize(txt)

In [15]:
def KeyWord(lst, search_terms):
    if search_terms in lst: 
        return 1 
    else:
        return 0 
    

    